## Matrix 합치기
- FASTQ로부터 align 한 다음 htseq count 한 raw count를 받아보자
- Single end : Fibula, Mandible/Maxilla/Tibia, ENCODE Osteocyte
- 우리 데이터 : "/data/project/OPLL/0.rnaraw/Analysis/~230316/00.PCA&Demographics/df/df_coladd.txt"


In [1]:
from Bio import SeqIO
from biomart import BiomartServer
import pandas as pd
import numpy as np
import os
import palettable

# Import R packages in Python
# base = importr('base')
# utils = importr('utils')
# deseq2 = importr('DESeq2')
# sva = importr ("sva")    # for Combat_seq

#from reComBat import reComBat

tabl = palettable.tableau.Tableau_20.mpl_colors

- GSE175236 (ENCODE Osteocyte),   GSE78608 (ENCODE Osteoblast), GSE227994 (Oxford_Ob)
- GSE220630 (Fibula, 5개), GSE149167 (maxilla, mandible, tibia)
- GSE188760 (OPLL, CSM, 2023, 4개), GSE69787(OPLL, Lig, 2016, 4개), GSE186542 (NP, 6개),
- GSE197172 (BM-MSC, 3개),  GSE102312 (BM-MSC_sb, 6개)

In [3]:
batch_list = []
batch_index = 0

# Display the filenames
for project in ["GSE175236", "GSE78608", "GSE227994", "GSE220630",  "GSE149167" , "GSE188760", "GSE69787", "GSE186542", "GSE197172", "GSE102312"]:
    DIR =  "/data/project/OPLL/01.bulkRNA/99.public/" + project + "/04.matrix"
    for filename in sorted(os.listdir(DIR))  :
        if os.path.isfile (os.path.join ( DIR, filename)) == False:
            continue
        if (project =="GSE149167") :
            if ( filename.split("_")[1].split(".tsv")[0] in (["4", "5", "6"])  ):
                continue
            
        if ("tsv" in filename) & ("norm_counts" not in filename):
            INPUT_TSV = DIR + "/" + filename
            #print ("{}\t{}".format (filename, df.shape ) )
            SAMPLE_ID = filename.split(".")[0]

            df = pd.read_csv (INPUT_TSV, sep = "\t", names = ["GENE_SYMBOL", SAMPLE_ID] )

            batch_list.append( batch_index )

            if len(batch_list) == 1:  # 맨 처음이라면
                df_concat = df
            else:
                df_concat = pd.merge (df_concat, df,  how = "outer").fillna (0)


    batch_index = batch_index + 1

In [12]:
# # GSE220630 (Fibula) : single end 라서 어쩔 수 없이 그냥 matrix를 합쳐주자

# import pandas as pd
# import numpy as np
# import pyensembl

# pyensemble = pyensembl.EnsemblRelease(77)   # release 77 uses human reference genome GRCh38

# INPUT_TSV = "/data/project/OPLL/0.rnaraw/99.public/GSE220630/03.matrix/GSE220630_norm_counts.tsv"
# df = pd.read_csv (INPUT_TSV, sep = "\t").iloc[ : , [0, 1, 6, 13, 19] ]  # 4개만 골라주자
# df.columns = ["ENSEMBL_ID", "Fibula_1", "Fibula_2", "Fibula_3", "Fibula_4"]
# df ["GENE_SYMBOL"] = ""

# for i in range (df.shape[0]):
#     E_ID = df.iloc [i, 0]
#     try:
#         #print ( "{}\t{}".format(E_ID, pyensemble.gene_name_of_gene_id( E_ID ) ) )
#         df.iloc[i, -1] = pyensemble.gene_name_of_gene_id( E_ID )
#     except:
#         #print ( "{} : No Gene Symbol".format (E_ID))
#         print ( "", end = "")
    
#     if i % 5000 == 0:
#         print ("", end = " ")


# df_concat = pd.merge (df_concat, df ,   left_on = "GENE_SYMBOL", right_on = "GENE_SYMBOL", how = "left").fillna (0)
# # Reorder DataFrame
# cols = df_concat.columns.tolist()
# cols.insert(1, cols.pop(cols.index('ENSEMBL_ID')))
# df_concat = df_concat[cols] 
# df_concat = df_concat.iloc[:-5, ]

# # Update batch_list
# batch_list = batch_list + [batch_index] * 4
# batch_index = batch_index + 1


### GTEX의 whole blood data 4개를 합쳐주자

In [5]:
df = pd.read_csv ( "/data/project/OPLL/01.bulkRNA/99.public/GTEX/gene_reads_2017-06-05_v8_whole_blood.txt",  sep = "\t")

NUM_SELECT_GTEX = 4
df = df.iloc [:, [2] + list( range(3, 3 + NUM_SELECT_GTEX)) ]
df.columns = ["GENE_SYMBOL"] + ["WB_{}".format(i) for i in range(1, df.shape[1])]
#df.head()
df_concat = pd.merge (df_concat, df ,   left_on = "GENE_SYMBOL", right_on = "GENE_SYMBOL", how = "left").fillna (0)

batch_list = batch_list + [batch_index] * NUM_SELECT_GTEX
batch_index = batch_index + 1

In [6]:
print (df_concat.shape, len(batch_list))

(26866, 53) 52


## Our data (local R에서 demographics 돌린 다음, DASH 에 올리자)

In [8]:
df = pd.read_csv ( "/data/project/OPLL/01.bulkRNA/Analysis/~230512/00.PCA&Demographics/df/df_coladd.txt",  sep = "\t")
coldata = [ int(line.rstrip('\n')) for line in open("/data/project/OPLL/01.bulkRNA/Analysis/~230512/00.PCA&Demographics/coldata/group.txt", 'r')]
batch_list = batch_list +  list ( coldata + np.max(batch_list) )

print ("Out data 개수 : {}\nbatch_list : {} ({}개)".format (len(coldata) , batch_list, len(batch_list)))

df_concat = pd.merge (df_concat, df ,   left_on = "GENE_SYMBOL", right_on = "GENE_SYMBOL", how = "left").fillna (0)

df_concat.iloc[:, 1:] = df_concat.iloc[:, 1:].astype(int)

Out data 개수 : 33
batch_list : [0, 0, 0, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 8, 8, 8, 9, 9, 9, 9, 9, 9, 10, 10, 10, 10, 12, 11, 12, 11, 12, 11, 12, 11, 12, 11, 12, 11, 12, 11, 12, 11, 12, 11, 12, 12, 13, 11, 12, 12, 13, 11, 12, 12, 11, 11, 12, 13, 11] (85개)


In [9]:
#df_concat.drop(["ENSEMBL_ID"], axis = 1).to_csv ("/data/project/OPLL/0.rnaraw/99.public/df_concat.tsv", sep = "\t", index = False, header = True)
df_concat.iloc[:-5, : ].to_csv ("/data/project/OPLL/01.bulkRNA/99.public/df_concat.tsv", sep = "\t", index = False, header = True)
pd.DataFrame(batch_list).to_csv ("/data/project/OPLL/01.bulkRNA/99.public/batch_list.tsv", sep = "\t", index = False, header = False)

In [11]:
df_concat.iloc[:,50:]

,WB_2,WB_3,WB_4,220605-LAMINA-1,220605-OLF-1,220629-LAMINA,220629-OLF,221018-LAMINA,221018-OLF,221102-LAMINA,...,230222-LAMINA,230222-LF,230222-OLF,230222-SPINOUS,230316-BODY,230316-OALL,230316-OPLL,230512-LAMINA,230512-LF,230512-OLF
0,102,138,113,278,279,241,600,99,227,462,...,393,174,789,105,658,1159,483,228,143,766
1,96,99,155,78,106,362,65,35,81,85,...,54,60,6,0,55,1,12,10,1,21
2,3,1,3,19,1,985,2,0,0,15,...,0,0,2,0,77,3,2,1,0,2
3,168,186,75,54386,84964,5514,92581,18803,70624,75325,...,14897,79849,21697,7313,53976,37127,67448,50025,55551,80234
4,5,74,75,109,148,16,88,99,73,85,...,10,78,11,0,68,26,49,52,65,67
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26861,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26862,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26863,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26864,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
# 전체 raw read count의 총합 (data 총량)
t = pd.DataFrame ( np.sum(df_concat.iloc[ : ,1 : ], axis = 0)  )
t.astype ("int").T

,ENCODE_Os_1,ENCODE_Os_2,ENCODE_Os_3,ENCODE_Ob-1,ENCODE_Ob-2,OB_Oxford_1,OB_Oxford_2,OB_Oxford_3,OB_Oxford_4,fibula_1,...,230222-LAMINA,230222-LF,230222-OLF,230222-SPINOUS,230316-BODY,230316-OALL,230316-OPLL,230512-LAMINA,230512-LF,230512-OLF
0,18159573,17574228,17650340,388902241,371095824,7509596,17158006,8147490,35935584,36755561,...,38549733,90915229,80443378,32207492,69336040,72545796,85412134,66262858,91904150,88673368
